In [1020]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\fefif\Desktop\PhD\Progetti\MatrixProductBP.jl`


In [1021]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra, TensorTrains
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using SparseArrays
include("convolve_fourier.jl");

In [1022]:
T = 4
N = 5
k = 4
seed = 1
rng = MersenneTwister(seed)
A = [0 1 1 1 1; 1 0 0 0 0; 1 0 0 0 0; 1 0 0 0 0; 1 0 0 0 0]
g = IndexedBiDiGraph(A)
# rand!(rng, A.nzval, (-1,1))  # fill directed incidence matrix with rand(-1,1)
β = 1.0
J = 1
h = 0.0
m⁰ = 1.0
K = 80

A

5×5 Matrix{Int64}:
 0  1  1  1  1
 1  0  0  0  0
 1  0  0  0  0
 1  0  0  0  0
 1  0  0  0  0

In [1023]:
w = [fill(IntegerGlauberFactor([J*A[j,i] for j in inneighbors(g,i)], h, β), T+1) for i in vertices(g)]
ϕᵢ = [t == 0 ? [(1-m⁰)/2, (1+m⁰)/2] : ones(2) for t in 0:T]
bp = mpbp(g, w, fill(2,N), T)#; ϕ=fill(ϕᵢ,N))

for μ in bp.μ
    for a in μ
        a.=rand.()
    end
end
;

In [1024]:
iters, cb = iterate!(bp, maxiter=2)

(2, CB_BP{ProgressMeter.ProgressUnknown, MatrixProductBP.var"#153#156"}(ProgressMeter.ProgressUnknown(false, false, 0, ProgressMeter.ProgressCore(:green, "Running MPBP: iter", 0.1, true, 0, VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(Base.Libc.WindowsRawSocket(0x00000000000002d0) open, 0 bytes waiting)), VSCodeServer.io_send_callback), false, 1, 2, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 0, 143870990379)), 0, 1, false, false, 1.733501425411e9, 1.733501425411e9, 1.733501425411e9)), [[[1.5, 1.5, 1.5, 1.5, 1.5], [1.5, 1.5, 1.5, 1.5, 1.5], [1.5, 1.5, 1.5, 1.5, 1.5], [1.5, 1.5, 1.5, 1.5, 1.5], [1.5, 1.5, 1.5, 1.5, 1.5]], [[1.3524257741140324, 1.4618255379588558, 1.5210438864514457, 1.710856950891432, 1.4425977290961092], [1.4522614637989797, 1.325591751454596, 1.4631559968072063, 1.3311714

In [1025]:
# bp_fourier = deepcopy(bp)
# iterate!(bp,maxiter=1)
# iterate_fourier!(bp_fourier,K, maxiter=1)
# ;

In [1026]:
bp_fourier = deepcopy(bp)
onebpiter!(bp, 1)
onebpiter_fourier!(bp_fourier, 1, K)

In [1027]:
# node = 4
# time = 3

# belief = bp.b[node]
# belief_fourier = bp_fourier.b[node]

# marg = marginals(belief)[time]
# marg_fourier = marginals(belief_fourier)[time]
# ;

In [1028]:
# display(marg)
# display(marg_fourier);

In [1029]:
import MatrixProductBP: marginalize
function marginalize(A::MPEM2{F}, var) where F
    if var==1
        return MPEM1{F}([@tullio b[m, n, xi] := a[m, n, xi, xj] for a in A]; z = A.z)
    elseif var==2
        return MPEM1{F}([@tullio b[m, n, xj] := a[m, n, xi, xj] for a in A]; z = A.z)
    end
end;

In [ ]:
node = 2    # messages are the same for every node!!!
time = 1

ed = inedges(bp.g, node) |> first
@show ed
μ = bp.μ[idx(ed)]
μ_marg = marginalize(μ,2)

ed_fourier = inedges(bp_fourier.g, node) |> first
@show ed_fourier
μ_fourier = bp_fourier.μ[idx(ed_fourier)]
μ_fourier_marg = marginalize(μ_fourier,2)

marg = marginals(μ_marg)[time]
marg_fourier = marginals(μ_fourier_marg)[time]
;

ed = Indexed Edge 1 => 2 with index 1
ed_fourier = Indexed Edge 1 => 2 with index 1


In [1045]:
display(marg)
display(marg_fourier);

2-element Vector{Float64}:
 0.49999999999999983
 0.5000000000000001

2-element Vector{Float64}:
 0.5071143385880872
 0.4928856614119129